# 评价类模型
## 层次分析法
1.算法原理分析
- 层次分析法是一种主观赋值评价方法也是一个多指标综合评价算法，常用于综合评价类模型。层次分析法将与决策有关的元素分解成目标、准则、方案等多个层次，并在此基础上进行定性和定量分析，是一种简单、实用的算法。

- 原理：是在分析一个现象或问题之前，首先将现象或问题根据他们的性质分解为相关因素，并依据因素之间的关系形成一个多层次的结构模型。然后通过经验或专家来判断低层元素对高层元素的相对重要性，并根据重要性的程度得出权重排序。

2.AHP层次分析法过程

- 分析系统中各因素之间的关系，建立系统的递阶层次结构。

- 对于同一层次的个元素关于上一层次中某一准则的重要性两两比较，构造两两比较矩阵（判断矩阵）。

- 由判断矩阵计算被比较元素对于该准则的相对权重，并进行一致性检验（检验通过权重才能用）。

- 填充权重矩阵，根据矩阵计算得分，得出结果。

## 构建层次结构模型
- 将所有相关因素分为目标层、准则层和方案层
![alt text](image-17.png)

## 构造判断矩阵
- 把准则层的指标进行两两判断，确定各准则层对目标层的权重
- 对于准则层A我们可以构建：
![alt_text](image-21.png)
- 矩阵里面写什么简单来说就是假如有三个元素A、B、C，A和B相比A重要多少，A比C中要多少，B比C中要多少，这个“多少”我们通常使用Santy的1-9标度方法给出，即：
![alr_text](image-22.png)

## 判断矩阵一致性检验
- 概念：
1. 矩阵中的所有元素均大于0且满足$A_{ij}*A_{ji}=1$,即正互反矩阵
2. 在正互反矩阵中满足$A_{ij}*A_{ji}=A_{ik}$,即一致矩阵
- A为一致矩阵的充要条件为：
1. $a_{ij}>0$
2. $a_{11}=a_{22}=...=a_{nn}=1$
3. $[a_{i1},a_{i2},...,a_{im}]=k_i[a_{11},a_{22},...,a_{1n}]$

## 计算一致性指标CI
$CI={\lambda_{max}-n\over n-1}$(n为矩阵的阶，$\lambda$为矩阵的最大特征值)

In [1]:
import numpy as np
matrix = np.array([[1, 1/7,1/4,1/6,2], 
              [7,1,3,3,5], 
              [4,1/3,1,1/4,6], 
              [6,1/3,4,1,8],
              [1/2,1/5,1/6,1/8,1]
              ])
n=matrix.shape[0]
eigen, _= np.linalg.eig(matrix)
lamb=np.max(np.real(eigen))
CI=(lamb-n)/(n-1)

## 计算一致性比例CR
1. 查找对应的平均随机一致性指标RI（见下方程序表）
2. 计算一致性比例$CR={CI\over RI}$
3. 如果CR<0.1,则可认为判断矩阵的一致性可以接受，否则需要对判断矩阵进行修正。

In [2]:
#ri值表
ri_values={1:0.00,
           2:0.00,
           3:0.52,
           4:0.89,
           5:1.12,
           6:1.26,
           7:1.36,
           8:1.41,
           9:1.46,
           10:1.49,
           11:1.52,
           12:1.54,
           13:1.56,
           14:1.58,
           15:1.59}
# CR计算函数
def CalCR(ri_values,matrix):
    n=matrix.shape[0]
    eigen, _= np.linalg.eig(matrix)
    lamb=np.max(np.real(eigen))
    CI=(lamb-n)/(n-1)
    RI=ri_values[n]
    CR=CI/RI
    if CR>=0.1:
        flag=0
    else:
        flag=1
    return CI,RI,CR,flag
CI,RI,CR,flag=CalCR(ri_values,matrix)
if flag==1:
    print(f"通过一致性检验,CR= {CR}")
else:
    print(f"未通过致性检验,请对判断矩阵进行修正,CR= {CR}")

未通过致性检验,请对判断矩阵进行修正,CR= 0.10036434427105571


## 合成权重
- 权重合成的方法有三种：算术平均值法、几何平均值法、特征值法

In [5]:
#算术平均值法
matrix1 = np.array([
    [1, 1/5, 1/4, 1/6, 2],
    [5, 1, 3, 3, 7],
    [4, 1/3, 1, 1/4, 6],
    [6, 1/3, 4, 1, 8],
    [1/2, 1/7, 1/6, 1/8, 1]
])
def arithmatic_mean(matrix):
    #讲判断矩阵按照列归一化
    matrix_norm=matrix / np.sum(matrix, axis=0)
    #归一化之后的矩阵按照行求和
    matrix_sum=np.sum(matrix_norm, axis=1)
    #相加后得到的矩阵再次归一化
    matrix_weight=matrix_sum / np.sum(matrix_norm)
    return matrix_weight
print("算数平均值法",arithmatic_mean(matrix1))
#几何平均值法
def geometric_mean(matrix):
    n = matrix.shape[0]
    #按照行相乘得到新的列向量
    matrix_row = np.prod(matrix, axis=1)
    #新的向量开n次方
    matrix_pow = np.power(matrix_row, 1/n)
    #归一化
    matrix_weight=matrix_pow / np.sum(matrix_pow)
    return matrix_weight
print("几何平均值法",geometric_mean(matrix1))
#特征值法
def eigen_mean(matrix):
    n = matrix.shape[0]
    #求出矩阵A的最大特征值以及其对应的特征向量
    eigen_values, eigen_vector = np.linalg.eig(matrix)
    matrix_pro = eigen_vector[:, np.argmax(eigen_values)]
    #归一化
    matrix_weight = matrix_pro / np.sum(matrix_pro)
    return matrix_weight
print("特征值法（不含虚数）",np.real(eigen_mean(matrix1)))
print("特征值法（含虚数）",eigen_mean(matrix1))

算数平均值法 [0.06197314 0.42186268 0.16643175 0.3116555  0.03807693]
几何平均值法 [0.06024609 0.43173896 0.15695138 0.31390275 0.03716082]
特征值法（不含虚数） [0.05692583 0.44076563 0.15197342 0.31475631 0.03557881]
特征值法（含虚数） [0.05692583+0.j 0.44076563+0.j 0.15197342+0.j 0.31475631+0.j
 0.03557881+0.j]


## 一种调整AHP不一致判断矩阵的优化方法
- 一般而言,判断矩阵的一致性改进目标有两个:
1. 使得改进后的判断矩阵满足一致性要求;
2. 尽量保证对原始判断矩阵的修改不违背专家的初衷.
3. 这两个目标综合在一起,可以理解为这样的一个带约束条件的优化问题:在保证判断矩阵满足一致性要求下,调整判断矩阵的元素值,使得调整后判断矩阵与原始判断矩阵的差异程度最小.
- 理论基础
该方法在打分时,主要根据资料数据、专家意见和评价主体经验,对评价要素之间取1∼9之间的整数或者是1∼9之间的整数的倒数.然而,由于决策问题的复杂性和人们认识的多样性,以及每个判断的片面性,要求每一个判断都具有一致性,显然是不可能的.从判断矩阵的分析看,这种整数赋值的方法,本身也是导致出现判断矩阵不一致的重要原因.
- 

## 层次分析法优点
- 层次分析法是一种系统性的分析方法。层次分析法把研究对象作为一个系统，按照分解、比较判断、综合的思维方式进行决策，成为继机理分析、统计分析之后发展起来的系统分析的重要工具。

- 层次分析法是一种简洁实用的决策方法。这种方法既不单纯追求高深数学，还不片面地注重行为、逻辑、推理，而是把定性方法与定量方法有机地结合起来。

- 层次分析法所需定量数据信息比较少。层次分析法主要是从评价者对评价问题的本质、要素的理解出发，比一般的定量方法更讲究定性的分析和判断。

## 层次分析法缺点
- 层次分析法指标过多时数据统计量大，且权重难以确定。

- 层次分析法的特征值和特征向量的精确求法比较复杂。在求判断矩阵的特征值和特征向量时，所用的方法和我们多元统计所用的方法是一样的。

- 层次分析法的定量数据较少，定性成分多，不易令人信服。

- 层次分析法只能从原有方案中进行选取，而不能为决策者提供解决问题的新方案。

参考文献：

[1]吴诗辉 刘晓东 贺波 解江.一种调整AHP不一致判断矩阵的优化方法[J].控制与决策,2016,31(11):2106-2112